# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [68]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [69]:
# Reaing the csv file
cities_csv = "../Weatherpy/output_data/weather_city_data.csv"
weather_cities = pd.read_csv(cities_csv)
weather_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.1203,-134.9692,76.08,71,100,10.11,pf,1619723338
1,batagay,67.6333,134.6333,19.67,97,99,2.08,ru,1619723338
2,shunyi,40.1250,116.6453,51.01,65,0,2.24,cn,1619723339
3,mount isa,-20.7333,139.5000,62.60,48,0,1.14,au,1619723339
4,jadu,31.9530,12.0261,80.73,11,5,4.36,ly,1619723340
...,...,...,...,...,...,...,...,...,...
547,sistranda,63.7256,8.8340,39.20,70,20,9.22,no,1619723589
548,masuguru,-11.3667,38.4167,72.37,96,10,2.77,tz,1619723589
549,nha trang,12.2500,109.1833,77.00,83,20,3.44,vn,1619723589
550,kochki,54.3358,80.4806,45.72,83,10,11.32,ru,1619723590


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [70]:
# Configuring the gmaps
gmaps.configure(api_key=g_key)

In [71]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_cities[["Lat","Lng"]].astype(float)

# Convert Poverty Rate to float and store
humidity = weather_cities["Humidity"].astype(float)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [72]:
# DataFrame to find ideal weather condition
# A max temperature lower than 80 degrees but higher than 70.

ideal_weather = weather_cities.loc[
    (weather_cities["Max Temp"] >=70) & 
    (weather_cities["Max Temp"] <=80)&
    (weather_cities["Wind Speed"] <=10)&
    (weather_cities["Cloudiness"] == 0)]

len(ideal_weather)


# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
ideal_weather= ideal_weather.dropna()

In [73]:
ideal_weather.reset_index()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,12,saint-philippe,-21.3585,55.7679,77.00,83,0,8.05,re,1619723339
1,42,hong gai,20.9560,107.0946,71.60,88,0,1.14,vn,1619723355
2,120,colonia,-34.1667,-57.5000,73.00,60,0,8.05,uy,1619723385
3,147,razole,16.4833,81.8333,76.37,80,0,0.54,in,1619723396
4,170,karratha,-20.7377,116.8463,79.59,40,0,9.15,au,1619723408
5,236,paso de carrasco,-34.8603,-56.0522,73.40,49,0,6.91,uy,1619723438
6,290,delta del tigre,-34.7633,-56.3853,73.40,53,0,4.61,uy,1619723466
7,392,chuy,-33.6971,-53.4616,70.36,47,0,3.44,uy,1619723512
8,436,boueni,-12.9025,45.0761,78.80,94,0,1.14,yt,1619723534


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [74]:
# Store into variable named hotel_df.
hotel_df = ideal_weather.reset_index()

In [75]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

In [76]:
# Set parameters to search for hotels with 5000 meters.
locations.head()
# dataframe with columns ('latitude', 'longitude', 'magnitude')
fig = gmaps.figure(map_type='HYBRID')
heatmap_layer = gmaps.heatmap_layer(locations)
fig.add_layer(heatmap_layer)

fig





Figure(layout=FigureLayout(height='420px'))

In [77]:
# Hit the Google Places API for each city's coordinates.

# geocoordinates
target_coordinates = locations
target_search = "hotel"
target_radius = 5000


# set up a parameters dictionary
params = {
    "locations": target_coordinates,
    "hotel": target_search,
    "radius": target_radius,
    "key": g_key
}


# Looping through all t information retrieving
for index,row in cities_clean.iterrows():
    curr_city= row["City"]
    curr_country= row["Country"]

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

NameError: name 'json' is not defined

In [78]:
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 5


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
